### Import Functions

In [1]:
from ipynb.fs.full.utilities import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
import pickle
from sklearn.metrics import classification_report

### Data Prep

In [2]:
X_train_60 = load_json('../../data/Train_60_X.json')
y_train_60 = load_json('../../data/Train_60_y.json')
X_dev = load_json('../../data/Dev_X.json')
y_dev = load_json('../../data/Dev_y.json')

In [3]:
verify_prepped_data(X_train_60, y_train_60)

In [4]:
verify_prepped_data(X_dev, y_dev)

In [5]:
X_train_60_flat = [word for sentence in X_train_60 for word in sentence]
y_train_60_flat = [label for sentence in y_train_60 for label in sentence]
X_dev_flat = [word for sentence in X_dev for word in sentence]
y_dev_flat = [label for sentence in y_dev for label in sentence]

In [6]:
len(X_train_60_flat) == len(y_train_60_flat)

True

In [7]:
len(X_dev_flat) == len(y_dev_flat)

True

In [8]:
X_train_60[0]

[{'word': '^',
  'sent_len': 6,
  'upos': 'PART',
  'first_word_in_sent': 'нека',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Te',
  'BOS': True,
  'next_word': 'нека',
  'next_word_upos': 'PART',
  'next_word_xpos_type': 'Tv',
  'word_after_next_word': 'не',
  'word_after_next_word_upos': 'PART',
  'word_after_next_word_xpos_type': 'Tn',
  'contains_publicistic_word': False},
 {'word': 'нека',
  'sent_len': 6,
  'upos': 'PART',
  'first_word_in_sent': 'нека',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Tv',
  'prev_word': '^',
  'prev_word_upos': 'PART',
  'prev_word_xpos_type': 'Te',
  'next_word': 'не',
  'next_word_upos': 'PART',
  'next_word_x

In [9]:
X_dev[0]

[{'word': '^',
  'sent_len': 42,
  'upos': 'CCONJ',
  'first_word_in_sent': 'но',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'punct',
  'BOS': True,
  'next_word': 'но',
  'next_word_upos': 'CCONJ',
  'next_word_xpos_type': 'Cc',
  'word_after_next_word': 'синовете',
  'word_after_next_word_upos': 'NOUN',
  'word_after_next_word_xpos_type': 'Nc',
  'word_after_next_word_xpos_gender_number_article': 'mpd',
  'contains_publicistic_word': True},
 {'word': 'но',
  'sent_len': 42,
  'upos': 'CCONJ',
  'first_word_in_sent': 'но',
  'contains_interrogative_word': False,
  'contains_interrogative_particle': False,
  'contains_imperative_verb': False,
  'contains_repetitive_conj_before': False,
  'between_two_verbs': False,
  'word_xpos_type': 'Cc',
  'prev_word': '^',
  'prev_word_upos': 'CCONJ',
  'prev_word_xpos_type': 'pu

In [10]:
v = DictVectorizer()
X_train_60_svm = v.fit_transform(X_train_60_flat)
X_dev_svm = v.transform(X_dev_flat)

In [11]:
le = preprocessing.LabelEncoder()
y_train_60_svm = le.fit_transform(y_train_60_flat)
y_dev_svm = le.transform(y_dev_flat)

In [12]:
le.classes_

array(['', '!', '"', '",', '".', '(', ')', '),', ').', ',', '-', '.',
       '...', ':', ':"', ';', '?'], dtype='<U3')

### Random Forest Model

In [13]:
%%time
clf = RandomForestClassifier(n_jobs=-1)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 1d 21h 38min 18s, sys: 7min 20s, total: 1d 21h 45min 38s
Wall time: 4h 4min 15s


RandomForestClassifier(n_jobs=-1)

In [14]:
pkl_filename = '../../data/random_forest_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [15]:
labels = list(clf.classes_)

In [16]:
y_pred = clf.predict(X_dev_svm)

In [17]:
target_names = list(le.classes_)

In [18]:
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

                  0.954     0.998     0.975    240418
           !      0.667     0.118     0.200       373
           "      0.774     0.128     0.220      2243
          ",      0.761     0.447     0.563       848
          ".      0.833     0.067     0.123       300
           (      0.979     0.866     0.919      1693
           )      0.925     0.795     0.855      1003
          ),      0.945     0.577     0.716       385
          ).      0.976     0.919     0.947       271
           ,      0.926     0.614     0.738     18651
           -      0.852     0.111     0.197       935
           .      0.919     0.990     0.954     11435
         ...      1.000     0.014     0.028        70
           :      0.973     0.353     0.518       935
          :"      0.000     0.000     0.000        68
           ;      0.741     0.032     0.061       628
           ?      0.852     0.203     0.328       453

    accuracy              

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
target_names.remove('')

In [20]:
labels.remove(0)

In [21]:
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

              precision    recall  f1-score   support

           !      0.667     0.118     0.200       373
           "      0.774     0.128     0.220      2243
          ",      0.761     0.447     0.563       848
          ".      0.833     0.067     0.123       300
           (      0.979     0.866     0.919      1693
           )      0.925     0.795     0.855      1003
          ),      0.945     0.577     0.716       385
          ).      0.976     0.919     0.947       271
           ,      0.926     0.614     0.738     18651
           -      0.852     0.111     0.197       935
           .      0.919     0.990     0.954     11435
         ...      1.000     0.014     0.028        70
           :      0.973     0.353     0.518       935
          :"      0.000     0.000     0.000        68
           ;      0.741     0.032     0.061       628
           ?      0.852     0.203     0.328       453

   micro avg      0.921     0.665     0.772     40291
   macro avg      0.820   

In [22]:
actual_y_pred = le.inverse_transform(y_pred)
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)
print(punctuated_sentences[:10735])

но синовете им не умъртви, защото постъпи според писаното в закона в книгата на мойсей, където господ заповяда и каза бащите да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях. но когато ти говоря ще отворя устата ти и ти им кажи: така казва господ бог, който слуша, нека слуша а който не слуша нека не слуша защото са бунтовен дом. защото кой е по-голям този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели както в предишните дни, заявява господ. а той беше на задната част, заспал на възглавница. ефрем е като птица славата му ще отлети. и така, той си отиде от него на известно разстояние. не бягам при халдейците. и израилевите синове излязоха от египетската земя строени. а когато ахитофел видя, че съветът му не се изпълни оседла магарето си и стана, и отиде у дома си в своя град и нареди домашните си работи и се обеси. ако умре някое животно от добитъка, който можете да ядете който се допре до мършата му

#### Results: Great precision with a bit low recall

### Random Forest - Entropy instead of Gini

In [25]:
%%time
clf = RandomForestClassifier(n_jobs=-1, criterion="entropy")
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 1d 22h 39min 39s, sys: 9min 26s, total: 1d 22h 49min 6s
Wall time: 4h 15min 34s


RandomForestClassifier(criterion='entropy', n_jobs=-1)

In [26]:
pkl_filename = '../../data/random_forest_v2_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [27]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.951     0.998     0.974    240418
           !      0.721     0.118     0.203       373
           "      0.756     0.122     0.210      2243
          ",      0.764     0.415     0.538       848
          ".      0.778     0.070     0.128       300
           (      0.978     0.855     0.913      1693
           )      0.928     0.780     0.847      1003
          ),      0.937     0.577     0.714       385
          ).      0.976     0.915     0.945       271
           ,      0.928     0.583     0.716     18651
           -      0.868     0.106     0.189       935
           .      0.919     0.987     0.952     11435
         ...      1.000     0.014     0.028        70
           :      0.969     0.336     0.499       935
          :"      0.000     0.000     0.000        68
           ;      0.760     0.030     0.058       628
           ?      0.930     0.205     0.336       453

    accuracy              

In [28]:
actual_y_pred = le.inverse_transform(y_pred)
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)
print(punctuated_sentences[:10735])

но синовете им не умъртви, защото постъпи според писаното в закона в книгата на мойсей, където господ заповяда и каза бащите да не умират заради синовете и синовете да не умират заради бащите а всеки да умира за собствения си грях. но когато ти говоря ще отворя устата ти и ти им кажи: така казва господ бог, който слуша нека слуша а който не слуша нека не слуша, защото са бунтовен дом. защото кой е по-голям този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели както в предишните дни, заявява господ. а той беше на задната част, заспал на възглавница. ефрем е като птица славата му ще отлети. и така, той си отиде от него на известно разстояние. не бягам при халдейците. и израилевите синове излязоха от египетската земя, строени а когато ахитофел видя, че съветът му не се изпълни оседла магарето си и стана, и отиде у дома си в своя град и нареди домашните си работи и се обеси. ако умре някое животно от добитъка, който можете да ядете който се допре до мършата му 

#### Result: Almost similar to the gini. For some reason ?, ! and , get very low recall. Recall in general isn't the greatest. Maybe this means overfitting?

### Random Forest - set the parameters that control tree size

In [46]:
%%time
clf = RandomForestClassifier(n_jobs=-1, max_depth=200, max_leaf_nodes=100000, max_samples=0.2)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 8h 56min 4s, sys: 1min 53s, total: 8h 57min 58s
Wall time: 49min 11s


RandomForestClassifier(max_depth=200, max_leaf_nodes=100000, max_samples=0.2,
                       n_jobs=-1)

In [47]:
pkl_filename = '../../data/random_forest_v3_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [48]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.935     0.999     0.966    240418
           !      1.000     0.003     0.005       373
           "      0.947     0.032     0.062      2243
          ",      0.785     0.271     0.403       848
          ".      1.000     0.007     0.013       300
           (      0.990     0.784     0.875      1693
           )      0.946     0.644     0.766      1003
          ),      0.980     0.517     0.677       385
          ).      0.987     0.860     0.919       271
           ,      0.949     0.470     0.629     18651
           -      1.000     0.053     0.102       935
           .      0.923     0.929     0.926     11435
         ...      0.000     0.000     0.000        70
           :      0.994     0.164     0.281       935
          :"      0.000     0.000     0.000        68
           ;      1.000     0.003     0.006       628
           ?      0.882     0.033     0.064       453

    accuracy              

#### Result: Very low recall

### Random Forest - more samples and features with smaller tree

In [13]:
%%time
clf = RandomForestClassifier(n_jobs=-1, max_depth=200, max_leaf_nodes=100000, max_samples=0.5)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 15h 45min 42s, sys: 4min 43s, total: 15h 50min 26s
Wall time: 1h 33min 13s


RandomForestClassifier(max_depth=200, max_leaf_nodes=100000, max_samples=0.5,
                       n_jobs=-1)

In [14]:
pkl_filename = '../../data/random_forest_v4_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [15]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.937     0.999     0.967    240418
           !      1.000     0.016     0.032       373
           "      0.937     0.033     0.064      2243
          ",      0.795     0.288     0.423       848
          ".      1.000     0.023     0.046       300
           (      0.990     0.808     0.890      1693
           )      0.940     0.691     0.797      1003
          ),      0.976     0.538     0.693       385
          ).      0.992     0.871     0.927       271
           ,      0.948     0.492     0.647     18651
           -      0.968     0.064     0.120       935
           .      0.925     0.926     0.926     11435
         ...      0.000     0.000     0.000        70
           :      0.991     0.234     0.379       935
          :"      0.000     0.000     0.000        68
           ;      1.000     0.006     0.013       628
           ?      0.917     0.073     0.135       453

    accuracy              

#### Result: We get better recall with more samples.

### Random Forest - Take into account more features and use smaller trees

In [16]:
%%time
clf = RandomForestClassifier(n_jobs=-1, max_depth=100, max_leaf_nodes=100, max_features=0.5, max_samples=0.5)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 3h 42min 25s, sys: 1min 4s, total: 3h 43min 29s
Wall time: 21min 45s


RandomForestClassifier(max_depth=100, max_features=0.5, max_leaf_nodes=100,
                       max_samples=0.5, n_jobs=-1)

In [17]:
pkl_filename = '../../data/random_forest_v5_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [18]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.944     0.995     0.969    240418
           !      0.000     0.000     0.000       373
           "      0.000     0.000     0.000      2243
          ",      0.614     0.328     0.427       848
          ".      0.000     0.000     0.000       300
           (      0.972     0.699     0.813      1693
           )      0.871     0.647     0.743      1003
          ),      0.926     0.423     0.581       385
          ).      0.959     0.786     0.864       271
           ,      0.865     0.536     0.662     18651
           -      1.000     0.016     0.032       935
           .      0.920     0.990     0.953     11435
         ...      0.000     0.000     0.000        70
           :      0.755     0.237     0.361       935
          :"      0.000     0.000     0.000        68
           ;      0.000     0.000     0.000       628
           ?      0.776     0.589     0.670       453

    accuracy              

#### Results: For the classes where we have enough samples, we get good recall and precision faster.

In [19]:
%%time
clf = RandomForestClassifier(n_jobs=-1, max_depth=100, max_leaf_nodes=100, max_features=None, max_samples=0.5)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 7h 58min 3s, sys: 2min 28s, total: 8h 31s
Wall time: 45min 55s


RandomForestClassifier(max_depth=100, max_features=None, max_leaf_nodes=100,
                       max_samples=0.5, n_jobs=-1)

In [20]:
pkl_filename = '../../data/random_forest_v6_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [21]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.945     0.995     0.969    240418
           !      0.000     0.000     0.000       373
           "      0.000     0.000     0.000      2243
          ",      0.622     0.375     0.468       848
          ".      0.000     0.000     0.000       300
           (      0.936     0.720     0.814      1693
           )      0.868     0.642     0.738      1003
          ),      0.926     0.423     0.581       385
          ).      0.959     0.779     0.859       271
           ,      0.862     0.535     0.660     18651
           -      0.577     0.032     0.061       935
           .      0.925     0.983     0.953     11435
         ...      0.000     0.000     0.000        70
           :      0.809     0.227     0.354       935
          :"      0.000     0.000     0.000        68
           ;      0.000     0.000     0.000       628
           ?      0.706     0.759     0.732       453

    accuracy              

#### Result: No big difference including all of the features

In [25]:
%%time
clf = RandomForestClassifier(n_jobs=-1, max_depth=100, max_leaf_nodes=100, max_features=0.5, n_estimators=10)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 24min 53s, sys: 4.26 s, total: 24min 58s
Wall time: 2min 36s


RandomForestClassifier(max_depth=100, max_features=0.5, max_leaf_nodes=100,
                       n_estimators=10, n_jobs=-1)

In [26]:
pkl_filename = '../../data/random_forest_v7_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [27]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.944     0.996     0.969    240418
           !      0.000     0.000     0.000       373
           "      0.000     0.000     0.000      2243
          ",      0.676     0.315     0.430       848
          ".      0.000     0.000     0.000       300
           (      0.975     0.703     0.817      1693
           )      0.865     0.647     0.740      1003
          ),      0.926     0.423     0.581       385
          ).      0.964     0.786     0.866       271
           ,      0.869     0.532     0.660     18651
           -      1.000     0.032     0.062       935
           .      0.921     0.988     0.953     11435
         ...      0.000     0.000     0.000        70
           :      0.727     0.233     0.353       935
          :"      0.000     0.000     0.000        68
           ;      0.000     0.000     0.000       628
           ?      0.766     0.636     0.695       453

    accuracy              

#### Result: Almost the same result as with 100 trees

In [28]:
%%time
clf = RandomForestClassifier(n_jobs=-1, max_depth=200, max_leaf_nodes=1000, max_features=0.5, n_estimators=10)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 2h 17min 24s, sys: 17.1 s, total: 2h 17min 41s
Wall time: 13min 58s


RandomForestClassifier(max_depth=200, max_features=0.5, max_leaf_nodes=1000,
                       n_estimators=10, n_jobs=-1)

In [29]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.960     0.995     0.977    240418
           !      0.556     0.013     0.026       373
           "      0.724     0.125     0.214      2243
          ",      0.670     0.533     0.594       848
          ".      0.857     0.020     0.039       300
           (      0.978     0.834     0.900      1693
           )      0.890     0.790     0.837      1003
          ),      0.981     0.540     0.697       385
          ).      0.979     0.852     0.911       271
           ,      0.864     0.674     0.757     18651
           -      0.880     0.071     0.131       935
           .      0.931     0.991     0.960     11435
         ...      0.000     0.000     0.000        70
           :      0.887     0.428     0.577       935
          :"      0.000     0.000     0.000        68
           ;      0.800     0.006     0.013       628
           ?      0.799     0.735     0.766       453

    accuracy              

In [30]:
pkl_filename = '../../data/random_forest_v8_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [31]:
actual_y_pred = le.inverse_transform(y_pred)
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)
print(punctuated_sentences[:10735])

но синовете им не умъртви, защото постъпи според писаното в закона в книгата на мойсей, където господ заповяда и каза: бащите да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях. но когато ти говоря, ще отворя устата ти и ти им кажи: така казва господ бог, който слуша, нека слуша, а който не слуша, нека не слуша, защото са бунтовен дом. защото кой е по-голям този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели както в предишните дни, заявява господ. а той беше на задната част, заспал на възглавница. ефрем е като птица славата му ще отлети. и така, той си отиде от него на известно разстояние. не бягам при халдейците. и израилевите синове излязоха от египетската земя, строени. а когато ахитофел видя, че съветът му не се изпълни оседла магарето си и стана и отиде у дома си в своя град и нареди домашните си работи и се обеси. ако умре някое животно от добитъка, който можете да ядете, който се допре до мърш

#### Results: Best results so far.

In [32]:
%%time
clf = RandomForestClassifier(n_jobs=-1, max_depth=200, max_leaf_nodes=1000, max_features=None, n_estimators=10)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 4h 40min 48s, sys: 55.5 s, total: 4h 41min 43s
Wall time: 29min 16s


RandomForestClassifier(max_depth=200, max_features=None, max_leaf_nodes=1000,
                       n_estimators=10, n_jobs=-1)

In [33]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.961     0.994     0.977    240418
           !      0.714     0.027     0.052       373
           "      0.714     0.147     0.243      2243
          ",      0.675     0.532     0.595       848
          ".      0.545     0.020     0.039       300
           (      0.970     0.830     0.894      1693
           )      0.889     0.791     0.837      1003
          ),      0.951     0.556     0.702       385
          ).      0.983     0.845     0.909       271
           ,      0.856     0.686     0.762     18651
           -      0.871     0.058     0.108       935
           .      0.930     0.989     0.959     11435
         ...      0.000     0.000     0.000        70
           :      0.865     0.445     0.588       935
          :"      0.000     0.000     0.000        68
           ;      0.833     0.008     0.016       628
           ?      0.786     0.713     0.748       453

    accuracy              

In [34]:
actual_y_pred = le.inverse_transform(y_pred)
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)
print(punctuated_sentences[:10735])

но синовете им не умъртви, защото постъпи според писаното в закона в книгата на мойсей, където господ заповяда и каза: бащите да не умират заради синовете и синовете да не умират заради бащите, а всеки да умира за собствения си грях. но когато ти говоря, ще отворя устата ти и ти им кажи: така казва господ бог, който слуша, нека слуша, а който не слуша, нека не слуша, защото са бунтовен дом. защото кой е по-голям този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели както в предишните дни, заявява господ. а той беше на задната част, заспал на възглавница. ефрем е като птица славата му ще отлети. и така, той си отиде от него на известно разстояние. не бягам при халдейците. и израилевите синове излязоха от египетската земя, строени. а когато ахитофел видя, че съветът му не се изпълни оседла магарето си и стана и отиде у дома си в своя град и нареди домашните си работи и се обеси. ако умре някое животно от добитъка, който можете да ядете, който се допре до мърш

#### Result: Slightly better performance to have all features

In [38]:
%%time
clf = RandomForestClassifier(n_jobs=-1, max_depth=100, max_leaf_nodes=2000, max_features=None, n_estimators=10)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 7h 31min 24s, sys: 45.6 s, total: 7h 32min 9s
Wall time: 46min 9s


RandomForestClassifier(max_depth=100, max_features=None, max_leaf_nodes=2000,
                       n_estimators=10, n_jobs=-1)

In [39]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.964     0.994     0.979    240418
           !      0.710     0.059     0.109       373
           "      0.740     0.177     0.285      2243
          ",      0.671     0.546     0.602       848
          ".      0.889     0.027     0.052       300
           (      0.964     0.857     0.907      1693
           )      0.888     0.809     0.847      1003
          ),      0.969     0.561     0.711       385
          ).      0.975     0.852     0.909       271
           ,      0.859     0.705     0.775     18651
           -      0.808     0.090     0.162       935
           .      0.933     0.990     0.961     11435
         ...      0.000     0.000     0.000        70
           :      0.832     0.497     0.622       935
          :"      0.000     0.000     0.000        68
           ;      0.714     0.016     0.031       628
           ?      0.792     0.740     0.765       453

    accuracy              

In [41]:
pkl_filename = '../../data/random_forest_v9_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [45]:
%%time
clf = RandomForestClassifier(n_jobs=-1, max_depth=100, max_leaf_nodes=5000, max_features=None, n_estimators=10)
clf.fit(X_train_60_svm, y_train_60_svm)

CPU times: user 15h 52min 26s, sys: 2min 22s, total: 15h 54min 49s
Wall time: 1h 38min 29s


RandomForestClassifier(max_depth=100, max_features=None, max_leaf_nodes=5000,
                       n_estimators=10, n_jobs=-1)

In [46]:
labels = list(clf.classes_)
y_pred = clf.predict(X_dev_svm)
target_names = list(le.classes_)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))
target_names.remove('')
labels.remove(0)
print(classification_report(y_dev_svm, y_pred, labels=labels, target_names=target_names, digits=3))

/Users/mter0707/opt/anaconda3/envs/masters/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

                  0.966     0.994     0.980    240418
           !      0.712     0.113     0.194       373
           "      0.720     0.218     0.335      2243
          ",      0.698     0.548     0.614       848
          ".      0.833     0.033     0.064       300
           (      0.967     0.870     0.916      1693
           )      0.885     0.821     0.852      1003
          ),      0.941     0.582     0.719       385
          ).      0.987     0.871     0.925       271
           ,      0.858     0.723     0.785     18651
           -      0.734     0.097     0.172       935
           .      0.937     0.991     0.963     11435
         ...      0.000     0.000     0.000        70
           :      0.868     0.540     0.666       935
          :"      0.000     0.000     0.000        68
           ;      0.520     0.021     0.040       628
           ?      0.819     0.779     0.799       453

    accuracy              

In [48]:
pkl_filename = '../../data/random_forest_v10_model.pkl'
with open(pkl_filename, 'wb') as file:
    pickle.dump(clf, file)

In [50]:
actual_y_pred = le.inverse_transform(y_pred)
punctuated_sentences = scikit_punctuate(X_dev_flat, actual_y_pred)
print(punctuated_sentences[:10735])

но синовете им не умъртви, защото постъпи според писаното в закона в книгата на мойсей, където господ заповяда и каза: бащите, да не умират заради синовете и синовете, да не умират заради бащите, а всеки да умира за собствения си грях. но когато ти говоря, ще отворя устата ти и ти им кажи: така казва господ бог, който слуша, нека слуша, а който не слуша, нека не слуша, защото са бунтовен дом. защото кой е по-голям този, който седи на трапезата ли или онзи, който слугува? а след това ще се насели както в предишните дни, заявява господ. а той беше на задната част, заспал на възглавница. ефрем е като птица славата му ще отлети. и така, той си отиде от него на известно разстояние. не бягам при халдейците. и израилевите синове излязоха от египетската земя, строени. а когато ахитофел видя, че съветът му не се изпълни оседла магарето си и стана, и отиде у дома си в своя град и нареди домашните си работи и се обеси. ако умре някое животно от добитъка, който можете да ядете, който се допре до м

#### Result: Best results so far.